In [16]:
import math

# HMM Parameters
STATES = ['E', '5', 'I']
START_PROB = {'E': 1.0, '5': 0.0, 'I': 0.0}

TRANSITIONS = {
    'start': {'E': 1.0},
    'E': {'E': 0.9, '5': 0.1},
    '5': {'I': 1.0},
    'I': {'I': 0.9, 'end': 0.1}
}

EMISSIONS = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.0,  'G': 0.95, 'T': 0.0},
    'I': {'A': 0.4,  'C': 0.1,  'G': 0.1,  'T': 0.4}
}

def log(x):
    return float('-inf') if x == 0 else math.log(x)

def get_log_prob_of_a_given_path(path, sequence):
    log_prob = 0.0
    prev = 'start'

    for i, (state, symbol) in enumerate(zip(path, sequence)):
        trans = TRANSITIONS[prev].get(state, 0)
        emit = EMISSIONS[state].get(symbol, 0)
        if emit == 0:
            return float('-inf')  # log(0) case
        log_prob += log(trans) + log(emit)
        prev = state

    if prev == 'I':
        log_prob += log(TRANSITIONS[prev]['end'])

    return round(log_prob, 2)

def viterbi(sequence):
    V = [{}]
    path_tracker = {}

    for state in STATES:
        emit = EMISSIONS[state].get(sequence[0], 0)
        V[0][state] = log(START_PROB[state]) + log(emit) if emit > 0 else float('-inf')
        path_tracker[state] = [state]

    for t in range(1, len(sequence)):
        V.append({})
        new_tracker = {}

        for curr_state in STATES:
            max_prob = float('-inf')
            best_prev = None
            emit = EMISSIONS[curr_state].get(sequence[t], 0)

            if emit == 0:
                V[t][curr_state] = float('-inf')
                new_tracker[curr_state] = path_tracker.get(curr_state, []) + [curr_state]
                continue

            for prev_state in STATES:
                trans = TRANSITIONS[prev_state].get(curr_state, 0)
                if trans == 0 or V[t - 1][prev_state] == float('-inf'):
                    continue
                prob = V[t - 1][prev_state] + log(trans) + log(emit)
                if prob > max_prob:
                    max_prob = prob
                    best_prev = prev_state

            if best_prev is not None:
                V[t][curr_state] = max_prob
                new_tracker[curr_state] = path_tracker[best_prev] + [curr_state]
            else:
                V[t][curr_state] = float('-inf')
                new_tracker[curr_state] = path_tracker.get(curr_state, []) + [curr_state]

        path_tracker = new_tracker

    final_prob = float('-inf')
    final_state = None
    for state in STATES:
        if 'end' in TRANSITIONS[state]:
            prob = V[-1][state] + log(TRANSITIONS[state]['end'])
            if prob > final_prob:
                final_prob = prob
                final_state = state

    return path_tracker[final_state], round(final_prob, 2)

def main():
    seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
    given_path = "EEEEEEEEEEEEEEE5IIIIIIIIII"
    print("Example Sequence:", seq)
    print("Example Path:    ", given_path)
    log_prob = get_log_prob_of_a_given_path(given_path, seq)
    print("Log probability of given path:", log_prob)
    best_path, best_log_prob = viterbi(seq)
    print("Best path from Viterbi:       ", ''.join(best_path))
    print("Log probability of best path: ", best_log_prob)

if __name__ == "__main__":
    main()


Example Sequence: CTTCATGTGAAAGCAGACGTAAGTCA
Example Path:     EEEEEEEEEEEEEEE5IIIIIIIIII
Log probability of given path: -42.58
Best path from Viterbi:        EEEEEEEEEEEEEEEEEE5IIIIIII
Log probability of best path:  -41.22
